In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp -r /content/drive/MyDrive/esmini /content/esmini

In [5]:
!apt-get install -y xvfb x11-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-utils x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 12 newly installed, 0 to remove and 49 not upgraded.
Need to get 8,046 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.

In [6]:
import sys, os
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/xvfb -O ../xvfb
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'
!cp -r /content/drive/MyDrive/xvfb /content/xvfb
!chmod 755 /content/drive/MyDrive/xvfb

Starting virtual X frame buffer: Xvfb.


In [7]:
!/content/xvfb start
%cd /content/esmini
!chmod 755 bin/esmini
!./bin/esmini --window 60 60 400 200 --aa_mode 0 --fixed_timestep 0.1 --headless --osc ./resources/xosc/cut-in.xosc

/bin/bash: line 1: /content/xvfb: Permission denied
/content/esmini
esmini GIT REV: v2.40.3-0-9fa4bae8
esmini GIT TAG: v2.40.3
esmini GIT BRANCH: tags/v2.40.3^0
esmini BUILD VERSION: N/A - client build
Run simulation decoupled from realtime, with fixed timestep: 0.10
Generated seed 407249511
Loading ./resources/xosc/cut-in.xosc (v1.1)
0 variables
Unsupported geo reference attr: +no_defs
Unsupported object type: rail-pole - interpret as NONE
Unsupported object type: rail-pole - interpret as NONE
Unsupported object type: guide-post - interpret as NONE
Unsupported object type: guide-post - interpret as NONE
Loaded OpenDRIVE: ./resources/xosc/../xodr/e6mini.xodr
Expr ${250/3.6} = 250/3.6 = 69.4444444444
Expr ${$EgoSpeed / 3.6} = 108 / 3.6 = 30.0000000000
Expr ${BrakeCondition_HWT + _ + $HeadwayTime_Brake} = BrakeCondition_HWT + _ + 0.7 = BrakeCondition_HWT_0.7
7 parameters:
   TargetSpeedFactor = 1.2
   EgoSpeed = 108
   HeadwayTime_Brake = 0.7
   HeadwayTime_LaneChange = 0.4
   EgoStartS 

In [9]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 49.8 MB/s eta 0:00:00


In [10]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [12]:
%cd /content
!git clone https://github.com/patrickw16/sim_loop.git

/content
Cloning into 'sim_loop'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 303 (delta 5), reused 87 (delta 4), pack-reused 212 (from 1)
Receiving objects: 100% (303/303), 76.26 MiB | 27.29 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [22]:
import ctypes as ct
import sys
import os
import cv2
import numpy as np

from ultralytics import YOLO

xosc_path = "/content/esmini/resources/xosc/cut-in.xosc"
lib_path = "/content/esmini"

lib_paths = {
    "linux": os.path.join(lib_path, "bin/libesminiLib.so"),
    "linux2": os.path.join(lib_path, "bin/libesminiLib.so"),
    "darwin": os.path.join(lib_path, "bin/libesminiLib.dylib"),
    "win32": os.path.join(lib_path, "esminiLib.dll"),
}
se = ct.CDLL(lib_paths[sys.platform])

# Definition of SE_ScenarioObjectState struct
class SESpeedActionStruct(ct.Structure):
    _fields_ = [
        ("id", ct.c_int),                # id of object to perform action
        ("speed", ct.c_float),
        ("transition_shape", ct.c_int),  # 0 = cubic, 1 = linear, 2 = sinusoidal, 3 = step
        ("transition_dim", ct.c_int),    # 0 = distance, 1 = rate, 2 = time
        ("transition_value", ct.c_float),
    ]

class SELaneChangeActionStruct(ct.Structure):
    _fields_ = [
        ("id", ct.c_int),                # id of object to perform action
        ("mode", ct.c_int),              # 0 = absolute, 1 = relative (own vehicle)
        ("target", ct.c_int),            # target lane id (absolute or relative)
        ("transition_shape", ct.c_int),  # 0 = cubic, 1 = linear, 2 = sinusoidal, 3 = step
        ("transition_dim", ct.c_int),    # 0 = distance, 1 = rate, 2 = time
        ("transition_value", ct.c_float),
    ]

class SELaneOffsetActionStruct(ct.Structure):
    _fields_ = [
        ("id", ct.c_int),                # id of object to perform action
        ("offset", ct.c_float),
        ("max_lateral_acc", ct.c_float),
        ("transition_shape", ct.c_int),  # 0 = cubic, 1 = linear, 2 = sinusoidal, 3 = step
    ]

class SEImage(ct.Structure):
    _fields_ = [
        ("width", ct.c_int),
        ("height", ct.c_int),
        ("pixelSize", ct.c_int),
        ("pixelFormat", ct.c_int),
        ("data", ct.POINTER(ct.c_ubyte)),
    ]

# specify some function return and argument types (needed for the floats)
se.SE_SetCameraMode.argtypes = [ct.c_int]
se.SE_SaveImagesToFile.argtypes = [ct.c_int]
se.SE_GetObjectNumberOfCollisions.argtypes = [ct.c_int]
se.SE_SaveImagesToRAM.argtypes = [ct.c_bool]
se.SE_FetchImage.argtypes = [ct.c_void_p]
se.SE_CollisionDetection.argtypes = [ct.c_bool]
se.SE_FetchImage.restype = ct.c_int

#For screenshots
#SE_SaveImagesToFile(int nrOfFrames) --> for testing purposes
#SE_SaveImageToRAM(bool state) --> also try

#Custom camera in front of vehicle, e.g. sensor mount position:
# ./bin/esmini --window 60 60 800 400 --osc ./resources/xosc/slow-lead-vehicle.xosc --custom_camera 3,0,0.6,0,0

# specify some arguments and return types of useful functions
se.SE_StepDT.argtypes = [ct.c_float]
se.SE_GetSimulationTime.restype = ct.c_float

# initialize some structs needed for actions
lane_offset_action = SELaneOffsetActionStruct()
lane_change_action = SELaneChangeActionStruct()
speed_action = SESpeedActionStruct()
img = SEImage()

# initialize esmini with provided scenario
#se.SE_Init(sys.argv[1].encode('ascii'), 0, 1, 0, 0)

se.SE_Init(xosc_path.encode(), 0, 3, 0, 1) #3 -> no viewer, but image generated
se.SE_SetCameraMode(5) #first person view

#se.SE_SaveImagesToFile(3)
se.SE_SaveImagesToRAM(True)
se.SE_CollisionDetection(True)

# Load a model
model = YOLO("/content/sim_loop/best.pt")

# Known width of the object (e.g., a car width in meters)
KNOWN_WIDTH = 2.0  # Example width in meters
# Focal length of the camera (calibrated)
FOCAL_LENGTH = 45 * 800 / 36  # Example focal length in pixels

j = 0
flag_speed_action = False
flag_braking = False
while se.SE_GetQuitFlag() == 0 and se.SE_GetSimulationTime() < 17.0:
    flag = se.SE_FetchImage(ct.byref(img))
    coll_ego = se.SE_GetObjectNumberOfCollisions(0)
    if not flag:
        total_bytes = img.pixelSize * img.width * img.height
        img_data = np.ctypeslib.as_array(img.data, shape=(total_bytes,))
        img_array = img_data.reshape((img.height, img.width, img.pixelSize, ))

        img_array = np.flip(img_array, 0) # flip y axis
        #img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB) # change BGR to RGB
        results = model(img_array)
        results[0].save()

        # Iterate through the results and calculate distances
        for r in results:
            for box in r.boxes:
                cls = box.cls
                conf = box.conf
                if conf >= 0.3:
                    # Calculate the width of the bounding box in pixels
                    box_width = box.xyxy[0][2] - box.xyxy[0][0]
                    ego_box_left_edge = 800/2 - box_width/2
                    object_right_edge = box.xyxy[0][2]
                    # Calculate the distance
                    distance = (KNOWN_WIDTH * FOCAL_LENGTH) / box_width
                    if object_right_edge > ego_box_left_edge and distance < 30:
                        flag_braking = True

        if flag_braking and not flag_speed_action:
            print("Injecting speed action - brake")
            speed_action.id               = 0
            speed_action.speed            = 0.0
            speed_action.transition_shape = 0
            speed_action.transition_dim   = 1
            speed_action.transition_value = 7.0
            se.SE_InjectSpeedAction(ct.byref(speed_action))
            flag_speed_action = True

    se.SE_StepDT(0.1)
    j += 1


0: 320x640 1 cars, 243.6ms
Speed: 2.0ms preprocess, 243.6ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)
Injecting speed action - brake

0: 320x640 (no detections), 224.6ms
Speed: 2.8ms preprocess, 224.6ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 223.5ms
Speed: 4.1ms preprocess, 223.5ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 234.6ms
Speed: 4.2ms preprocess, 234.6ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 230.9ms
Speed: 2.8ms preprocess, 230.9ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 216.1ms
Speed: 2.7ms preprocess, 216.1ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 245.5ms
Speed: 3.4ms preprocess, 245.5ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 230.